In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
import nltk
tokenizer = nltk.RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
import seaborn as sns
import time
import re
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
random.seed(123)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
df_train = pd.read_csv("A.csv")
df_dev = pd.read_csv("A.csv")
df_test = pd.read_csv("A.csv")
# pour appliquer MLP sur Les données B
# df_train = pd.read_csv("B.csv")
# df_dev = pd.read_csv("B.csv")
# df_test = pd.read_csv("B.csv")

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
train_comments = df_train['commentaire'].values.tolist()

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(train_comments)]

In [ ]:
max_epochs = 10
vec_size = 100
alpha = 0.03

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(1,max_epochs+1):
#     if epoch%2==0:
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=max_epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_lstm.model")
print("Model Saved")

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
model = Doc2Vec.load("d2v_lstm.model")

In [ ]:
emb_dev = []
for i in range(len(df_dev)):
    vector = model.infer_vector(word_tokenize(df_dev['commentaire'][i])) 
    emb_dev.append(vector)


In [ ]:
df_train['doc2vec'] = [model.dv[i] for i in range(len(df_train))]
df_dev['doc2vec'] = [vector for vector in emb_dev]

In [ ]:
X_train = df_train['doc2vec']
X_dev = df_dev['doc2vec']

In [ ]:
y_train = df_train['note']
y_dev = df_dev['note']

In [ ]:
x_train = np.asarray(X_train.tolist())
x_train = np.expand_dims(x_train, -1)
# x_train

In [ ]:
x_train.shape

In [ ]:
x_dev = np.asarray(X_dev.tolist())
x_dev = np.expand_dims(x_dev, -1)
# x_train

In [ ]:
x_dev.shape

In [ ]:
num_classes = 10

In [ ]:
# Create the model
model = Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(100,1)))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
target_train=y_train.values-1
y_target_train=to_categorical(target_train, num_classes=10)
num_classes=y_target_train.shape[1]

In [ ]:
target_dev=y_dev.values-1
y_target_dev=to_categorical(target_dev, num_classes=10)
num_classes=y_target_train.shape[1]

In [ ]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint

st = time.time()

#create callback
filepath = 'my_best_model_mlp30.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]


history=model.fit(x_train, y_target_train, validation_data=(x_dev, y_target_dev),epochs=30, batch_size=128, verbose=1,callbacks=callbacks)

elapsed_time = time.time() - st
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
df_test.isnull().sum()

In [ ]:
emb_test = []
for i in range(len(df_dev)):
    vector = model.infer_vector(word_tokenize(df_test['commentaire'][i])) 
    emb_test.append(vector)
df_test['doc2vec'] = [vector for vector in emb_test]

In [ ]:
X_test = df_test['doc2vec']

In [ ]:
from keras.models import load_model
model = load_model('my_best_model_mlp30.hdf5')

In [ ]:
x_test = np.asarray(X_test)
x_test = np.expand_dims(x_test, -1)

In [ ]:
yhat = model.predict(x_test)

In [ ]:
yhat

In [ ]:
yhat.shape

In [ ]:
prediction = np.argmax(yhat,axis =1)

In [ ]:
np.unique(prediction)

In [ ]:
notes = [] #génération du fichier pour la plateforme d'évaluation
with open("sortie_my_best_model_mlp30_doc2vec.txt", "w") as f:
    for i in range(len(df_test['review_id'])):
        notes.append((prediction[i]+1)/2)
        f.write(str(df_test['review_id'].iloc[i])+' '+str(notes[i]).replace('.',',')+'\n')